In [ ]:
# langchain reads the OPENAI_API_KEY environment variable.
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY and len(OPENAI_API_KEY) > 32

In [ ]:
from homematch_functions import load_documents
from homematch_functions import load_vector_db
from homematch_functions import build_similar_documents
from homematch_functions import filter_similar_documents
from homematch_functions import personalize_listing

In [ ]:
documents = load_documents()
documents_by_uuid: dict[str, str] = {
    doc.metadata["record_uuid"]:doc.page_content for doc in documents
}
vector_db = load_vector_db(documents)
documents[0]

In [ ]:
questions = [
    "How big do you want your house to be?",
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",
]

In [ ]:
answers_1 = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters.",
]


In [ ]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider

form_row_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_rows = []
for index, question in enumerate(questions):
    form_rows.append(Box([
        Label(value=question), 
        Textarea(layout=Layout(height='4em', width='60%'), value=answers_1[index])
    ], layout=form_row_layout))

form = Box(form_rows, layout=Layout(
    display='flex',
    flex_flow='column',
    border='none',
    align_items='stretch',
    width='100%'
))

form

In [ ]:
similar_documents_ids = build_similar_documents(vector_db, answers_1)
filtered_documents_ids = filter_similar_documents(similar_documents_ids, 3)
filtered_documents_ids

In [ ]:
personalized_listings = {}
for record_uuid, answer_indexes in filtered_documents_ids.items():
    listing = documents_by_uuid[record_uuid]
    filtered_answers = [answers_1[i] for i in answer_indexes]
    result = personalize_listing(listing, filtered_answers)
    personalized_listings[record_uuid] = {
        "record_uuid": record_uuid,
        "filtered_answers": filtered_answers,
        "description": result,
    }
print(f"{len(personalized_listings)=}")

In [ ]:
from IPython.display import HTML
from homematch_index_html import get_data_urls

images = get_data_urls(list(personalized_listings.keys()))

blocks = []
for record_uuid, listing in personalized_listings.items():
    description = listing["description"].replace("\n", "<br>")
    blocks.append(f"""
        <div style="border: 2px solid gray; margin:2em; padding:1em;">
            {record_uuid}<br>
            <img src="{images[record_uuid]}" width="256" style="float:left; margin:1em;"/>
            {description}
        </div>
    """)

HTML('\n'.join(blocks))